### 1  a) parse the following JSON data stream

-[ ] are for JSON arrays, which are called list in Python
- { } are for JSON objects, which are called dict in Python

In [1]:
# The data is not in the correct json format, example, having single quotes instead of double quotes and is also having
# some other special characters. Replacing them and saving the file in correct format which can be parsed.


import json
import ast
import io
f = io.open("ff.txt",'r',encoding='utf-8')
lines = f.readlines()
# print(lines)
f.close()
lines = [line.replace("'", '"').replace('‘', '"').replace(' "', '"').replace('’', '"').replace('\n', ' ').replace('" ', '",') for line in lines]
# print (lines)

newLines = ''
for line in lines:
    newLines += str(line.strip('\n'))

f = io.open("ff2.txt",'w',encoding='utf-8')
f.write(u"%s" %(lines))
f.close()

jsonData = ast.literal_eval(newLines) #to convert text to meaningful expression


finaldata = {}
for i in range(len(jsonData)):
    finaldata[str(i)] = jsonData[i]

with io.open("final.json", 'w', encoding='utf8') as json_file:
    data = json.dumps(finaldata, ensure_ascii=False)
    json_file.write(u"%s" %(data))


In [2]:
jsonData

[{'name': 'Donna Summer',
  'updated': '2016-02-26T09:09:37',
  'recommendations': ['I worked with Donna for 5 years and can highly recommend her as an experienced, knowledgeable. She is very hardworking designer with a meticulous eye for detail and creativity',
   'I have worked with Donna for a number of years and have always found her to be extremely professional and a real expert in her field.',
   'Donna is always on the ball, highly organised and a real pleasure to work with.']},
 {'name': 'Justin Bieber',
  'updated': '2016-02-26T09:09:37',
  'recommendations': ['Justin is a consummate professional and a pleasure to work with at all times!']}]

In [3]:
jsonData[0]

{'name': 'Donna Summer',
 'updated': '2016-02-26T09:09:37',
 'recommendations': ['I worked with Donna for 5 years and can highly recommend her as an experienced, knowledgeable. She is very hardworking designer with a meticulous eye for detail and creativity',
  'I have worked with Donna for a number of years and have always found her to be extremely professional and a real expert in her field.',
  'Donna is always on the ball, highly organised and a real pleasure to work with.']}

In [4]:
jsonData[0]["recommendations"]

['I worked with Donna for 5 years and can highly recommend her as an experienced, knowledgeable. She is very hardworking designer with a meticulous eye for detail and creativity',
 'I have worked with Donna for a number of years and have always found her to be extremely professional and a real expert in her field.',
 'Donna is always on the ball, highly organised and a real pleasure to work with.']

In [5]:
jsonData[0]["recommendations"][2]

'Donna is always on the ball, highly organised and a real pleasure to work with.'

In [6]:
jsonData[1]["name"]

'Justin Bieber'

In [7]:
jsonData[1]["recommendations"]

['Justin is a consummate professional and a pleasure to work with at all times!']

## b) create an LDA, or other model of your choice, for the review data, and
##  c) convert the reviews into feature vectors using the model that you built.

### Combined Solutions below:

In [59]:
#Bag of words model is implemented.
# importing the packages for this part
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer


stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # remove non letters
    text = re.sub("[^a-zA-Z]", " ", text)
    # tokenize
    tokens = nltk.word_tokenize(text)
    # stem
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english',
    max_features = 85
)

In [ ]:
features_matrix = vectorizer.fit_transform(Doc_reviews) 
#converting the reviews into feature vectors of fixed size using bag of words model approach. 
##Even better, if TfidfVectorizer() was used instead of CountVectorizer(), because it 
##would have downweighted words that occur frequently across docuemnts.

In [ ]:
#Converting features Matrix to Pandas Dataframe to see the word frequencies.
doc_term_matrix = features_matrix.todense()

df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names())
df

### 2) Now, the team would like to search the reviews. Given a free-text string, called search_review, can you write another python script to compare the free-text string to all the review feature vectors in the database. The team would like to rank the comparisons and display the top three on the screen (python print).

In [8]:
a={'search_review': 'Excellent professional attitude that works across the whole company and is a pleasure to work with'}


In [9]:
a["search_review"]

'Excellent professional attitude that works across the whole company and is a pleasure to work with'

In [41]:
documents=[]

#assumption: The search review is appended at the very end. This is just for demonstration, hence didn't write a function.
#Ideally, the way data is given, depending on that a function will be written to append the data to documents array
# Here, just for demonstration, the three documents are parsed from the json example given in the assignment

doc1 = jsonData[0]["recommendations"][0]
doc2= jsonData[0]["recommendations"][1]
doc3= jsonData[0]["recommendations"][2]
doc4= a["search_review"]

documents.append(doc1)
documents.append(doc2)
documents.append(doc3)
documents.append(doc4)

In [14]:
documents

['Donna is always on the ball, highly organised and a real pleasure to work with.',
 'I have worked with Donna for a number of years and have always found her to be extremely professional and a real expert in her field.',
 'I worked with Donna for 5 years and can highly recommend her as an experienced, knowledgeable. She is very hardworking designer with a meticulous eye for detail and creativity',
 'Excellent professional attitude that works across the whole company and is a pleasure to work with']

In [15]:
# importing the packages for the 2nd part of the assignment

import gensim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)


# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

3.8.3


In [42]:
# Preparing a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Preparing the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences i.e. reviews into bag-of-words vectors.

def Vector_conversion(doc):
    vect= dictionary.doc2bow(simple_preprocess(doc))
    return vect

Vectors=[]
for i in documents:
    Vectors.append(Vector_conversion(i))
    print (i)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity_matrix` (Method will be removed in 4.0.0, use gensim.models.keyedvectors.WordEmbeddingSimilarityIndex instead).
  """


I worked with Donna for 5 years and can highly recommend her as an experienced, knowledgeable. She is very hardworking designer with a meticulous eye for detail and creativity
I have worked with Donna for a number of years and have always found her to be extremely professional and a real expert in her field.
Donna is always on the ball, highly organised and a real pleasure to work with.
Excellent professional attitude that works across the whole company and is a pleasure to work with


In [60]:
# Computing soft cosine similarity

def similarity(vec1, vec2):
    res= softcossim(vec1, vec2, similarity_matrix)
    return res
    
leng= len(Vectors)
dict1={}

for i in range(0,leng-1):
    sim=similarity(Vectors[leng-1], Vectors[i])
    print ("Similarity of new text with vector {} is ".format(i), sim)
    data = {"Similarity_with_Vector{}".format(i) : sim}
    dict1.update(data)
    
    

Similarity of new text with vector 0 is  0.4423087497953782
Similarity of new text with vector 1 is  0.5040128735857902
Similarity of new text with vector 2 is  0.6538491400382125


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  after removing the cwd from sys.path.


In [61]:
dict1

{'Similarity_with_Vector0': 0.4423087497953782,
 'Similarity_with_Vector1': 0.5040128735857902,
 'Similarity_with_Vector2': 0.6538491400382125}

In [71]:
#for sorting with respect to decresing order of similarity of the given string with other reviews 
# and giving the top three.

sort_orders = sorted(dict1.items(), key=lambda x: x[1], reverse=True)[:3]

for i in sort_orders:
    print(i[0], i[1])

Similarity_with_Vector2 0.6538491400382125
Similarity_with_Vector1 0.5040128735857902
Similarity_with_Vector0 0.4423087497953782
